In [1]:
import time
import asyncio
import pymongo
import json
from pymongo import MongoClient
from indy import anoncreds, did, ledger, pool, wallet, IndyError

async def run():
    print("Getting started -> started")

    # Set protocol version 2 to work with Indy Node 1.4
    await pool.set_protocol_version(2)

    pool_ = {
        'name': 'pool1',
        'config': json.dumps({"genesis_txn": '/home/indy/sandbox/pool_transactions_genesis'})
    }
    print("Open Pool Ledger: {}".format(pool_['name']))

    try:
        await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
    except IndyError as ex:
        if ex.error_code == 306:
            pass
    pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

    client = MongoClient(host='steward_db', port=27017, username='root', password='pass', authSource="admin")
    db = client["steward_db"]

    print("==============================")
    print("=== Getting Trust Anchor credentials for Faber, Acme, Thrift and Government  ==")
    print("------------------------------")

    steward = {
        'name': "Sovrin Steward",
        'wallet_config': json.dumps({'id': 'sovrin_steward_wallet'}),
        'wallet_credentials': json.dumps({'key': 'steward_wallet_key'}),
        'pool': pool_['handle'],
        'seed': '000000000000000000000000Steward1'
    }

    await create_wallet(steward)

    print("\"Sovrin Steward\" -> Create and store in Wallet DID from seed")
    steward['did_info'] = json.dumps({'seed': steward['seed']})
    steward['did'], steward['key'] = await did.create_and_store_my_did(steward['wallet'], steward['did_info'])

    db.steward_tb.insert_one(steward);

    print("==============================")
    print("== Getting Trust Anchor credentials - Government getting Verinym  ==")
    print("------------------------------")

    supplier = {
        'name': 'Supplier',
        'wallet_config': json.dumps({'id': 'supplier_wallet'}),
        'wallet_credentials': json.dumps({'key': 'supplier_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'TRUST_ANCHOR'
    }

    await getting_verinym(steward, supplier)
    print("Getting started done")

   

   
# async def steward_registration(steward):
#     await create_wallet(steward)
#     print("\"Sovrin Steward\" -> Create and store in Wallet DID from seed")
#     steward['did_info'] = json.dumps({'seed': steward['seed']})
#     steward['did'], steward['key'] = await did.create_and_store_my_did(steward['wallet'], steward['did_info'])


# async def supplier_registration(steward, supplier):
#     await getting_verinym(steward, supplier)
    

async def create_wallet(identity):
    print("\"{}\" -> Create wallet".format(identity['name']))
    try:
        await wallet.create_wallet(identity['wallet_config'], identity['wallet_credentials'])
    except IndyError as ex:
        if ex.error_code == 306:
            pass
    identity['wallet'] = await wallet.open_wallet(identity['wallet_config'], identity['wallet_credentials'])


async def getting_verinym(from_, to):
    await create_wallet(to)

    (to['did'], to['key']) = await did.create_and_store_my_did(to['wallet'], "{}")

    from_['info'] = {
        'did': to['did'],
        'verkey': to['key'],
        'role': to['role'] or None
    }

    await send_nym(from_['pool'], from_['wallet'], from_['did'], from_['info']['did'],
                   from_['info']['verkey'], from_['info']['role'])


async def send_nym(pool_handle, wallet_handle, _did, new_did, new_key, role):
    nym_request = await ledger.build_nym_request(_did, new_did, new_key, None, role)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, nym_request)

if __name__ == '__main__':
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    await run()
    time.sleep(1)  # FIXME waiting for libindy thread complete


Getting started -> started
Open Pool Ledger: pool1
=== Getting Trust Anchor credentials for Faber, Acme, Thrift and Government  ==
------------------------------
"Sovrin Steward" -> Create wallet
"Sovrin Steward" -> Create and store in Wallet DID from seed
== Getting Trust Anchor credentials - Government getting Verinym  ==
------------------------------
"Government" -> Create wallet


PoolLedgerTimeout: 